In [14]:
%matplotlib inline

In [15]:
import numpy as np
from sklearn.svm import SVR
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV

In [16]:
import tensorflow
tensorflow.keras.__version__

'2.4.0'

In [17]:
weather = pd.read_csv('sonomaDeepFake.csv')
weather.head()

,Unnamed: 0,winery,vintage,points,cities,MarchLowTemp,MarchHighTemp,MarchAvgTemp,MarchPrecip,MarchCloudiness,...,JulyAvgTemp,JulyPrecip,JulyCloudiness,JulyHumidity,AugLowTemp,AugHighTemp,AugAvgTemp,AugPrecip,AugCloudiness,AugHumidity
0,9,River Road,2011,87,Guttenberg,32.451613,44.290323,39.580645,8.6,40.838710,...,79.354839,6.6,20.387097,77.645161,67.225806,80.935484,75.419355,18.6,24.741935,79.451613
1,6,Novy,2009,87,Healdsburg,41.548387,60.677419,53.193548,5.7,26.129032,...,75.903226,0.0,2.580645,52.064516,55.451613,82.838710,73.935484,0.1,4.967742,54.612903
2,15,Barnett,2013,90,Saint Helena,72.935484,75.000000,73.806452,0.1,64.096774,...,68.741935,1.4,58.258065,73.580645,65.451613,70.322581,67.419355,0.8,81.612903,77.870968
3,17,En Garde,2014,90,Glen Ellen,24.870968,42.612903,35.451613,1.0,45.064516,...,72.483871,1.8,22.709677,77.129032,61.000000,80.129032,73.225806,4.2,38.838710,83.903226
4,19,MacRostie,2014,90,Healdsburg,42.322581,64.483871,56.322581,6.5,38.516129,...,76.709677,0.0,11.645161,60.838710,54.451613,83.161290,73.225806,0.1,8.322581,64.419355


In [18]:
weatherClean=weather.drop(["Unnamed: 0"], axis=1)
weatherClean.head()

,winery,vintage,points,cities,MarchLowTemp,MarchHighTemp,MarchAvgTemp,MarchPrecip,MarchCloudiness,MarchHumidity,...,JulyAvgTemp,JulyPrecip,JulyCloudiness,JulyHumidity,AugLowTemp,AugHighTemp,AugAvgTemp,AugPrecip,AugCloudiness,AugHumidity
0,River Road,2011,87,Guttenberg,32.451613,44.290323,39.580645,8.6,40.838710,78.548387,...,79.354839,6.6,20.387097,77.645161,67.225806,80.935484,75.419355,18.6,24.741935,79.451613
1,Novy,2009,87,Healdsburg,41.548387,60.677419,53.193548,5.7,26.129032,70.935484,...,75.903226,0.0,2.580645,52.064516,55.451613,82.838710,73.935484,0.1,4.967742,54.612903
2,Barnett,2013,90,Saint Helena,72.935484,75.000000,73.806452,0.1,64.096774,78.225806,...,68.741935,1.4,58.258065,73.580645,65.451613,70.322581,67.419355,0.8,81.612903,77.870968
3,En Garde,2014,90,Glen Ellen,24.870968,42.612903,35.451613,1.0,45.064516,71.225806,...,72.483871,1.8,22.709677,77.129032,61.000000,80.129032,73.225806,4.2,38.838710,83.903226
4,MacRostie,2014,90,Healdsburg,42.322581,64.483871,56.322581,6.5,38.516129,72.774194,...,76.709677,0.0,11.645161,60.838710,54.451613,83.161290,73.225806,0.1,8.322581,64.419355


In [19]:
# Data Pre-Processing

In [20]:
weatherClean.columns

Index(['winery', 'vintage', 'points', 'cities', 'MarchLowTemp',
       'MarchHighTemp', 'MarchAvgTemp', 'MarchPrecip', 'MarchCloudiness',
       'MarchHumidity', 'AprilLowTemp', 'AprilHighTemp', 'AprilAvgTemp',
       'AprilPrecip', 'AprilCloudiness', 'AprilHumidity', 'MayLowTemp',
       'MayHighTemp', 'MayAvgTemp', 'MayPrecip', 'MayCloudiness',
       'MayHumidity', 'JuneLowTemp', 'JuneHighTemp', 'JuneAvgTemp',
       'JunePrecip', 'JuneCloudiness', 'JuneHumidity', 'JulyLowTemp',
       'JulyHighTemp', 'JulyAvgTemp', 'JulyPrecip', 'JulyCloudiness',
       'JulyHumidity', 'AugLowTemp', 'AugHighTemp', 'AugAvgTemp', 'AugPrecip',
       'AugCloudiness', 'AugHumidity'],
      dtype='object')

In [21]:
weatherCleandf = weatherClean.drop(['winery', 'vintage', 'cities'], axis=1)

In [22]:
X = weatherCleandf.drop(['points'], axis=1).values.astype(float)
y = weatherCleandf['points'].values.astype(float)

In [26]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X = sc_X.fit_transform(X)
y = sc_y.fit_transform(y.reshape(-1,1))

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [28]:
C_range = 10. ** np.arange(-3, 8)
gamma_range = 10. ** np.arange(-5, 4)

param_grid = dict(gamma=gamma_range, C=C_range)

cv = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=42)
grid = GridSearchCV(SVR(), param_grid=param_grid, cv=cv)
grid.fit(X, np.ravel(y,order="a"))

print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'C': 1.0, 'gamma': 0.01} with a score of 0.07


In [39]:
regressor = SVR(C=1.0, kernel='rbf', gamma=0.01)
regressor.fit(X,np.ravel(y,order="a"))

SVR(gamma=0.01)

In [40]:
y_pred = regressor.predict(X_test)
y_pred = sc_y.inverse_transform(y_pred)

In [41]:
df = pd.DataFrame({'Real Values':sc_y.inverse_transform(y_test.reshape(-1)), 'Predicted Values':y_pred})
df

,Real Values,Predicted Values
0,-2.219063,-0.379382
1,0.682789,0.094295
2,-0.477952,-0.377865
3,-2.219063,-0.491735
4,0.102418,0.002009
5,-0.477952,-0.377969


In [42]:
training_score = regressor.score(X_train, y_train)
testing_score = regressor.score(X_test, y_test)

print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 0.09832391219054426
Testing Score: 0.06867449817010352
